In [ ]:
%pip install requests beautifulsoup4 tqdm
%pip install selenium webdriver-manager tqdm

In [ ]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


SAVE_DIR = "Data"
os.makedirs(SAVE_DIR, exist_ok=True)


options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

wait = WebDriverWait(driver, 30)

BASE_URL = "https://laws.moj.gov.sa/ar/legislations-regulations"
driver.get(BASE_URL)

time.sleep(8)


def download(url, name):

    r = requests.get(url, stream=True)

    path = os.path.join(SAVE_DIR, name)

    with open(path, "wb") as f:
        for chunk in r.iter_content(1024):
            f.write(chunk)


count = 0
MAX_PAGES = 200


print("🚀 Starting...\n")


for page in range(1, MAX_PAGES + 1):

    print(f"📄 Page {page}")

    items = wait.until(
        EC.presence_of_all_elements_located(
            (By.XPATH, "//a[contains(@href,'/ar/legislations')]")
        )
    )

    for item in items:

        try:
            driver.execute_script("arguments[0].scrollIntoView(true);", item)
            time.sleep(1)
            item.click()

            # استنى التاب الجديد
            wait.until(EC.number_of_windows_to_be(2))

            tabs = driver.window_handles

            driver.switch_to.window(tabs[1])

            time.sleep(3)

            pdf_url = driver.current_url

            if ".pdf" not in pdf_url.lower():
                print("⚠️ Not direct PDF:", pdf_url)
                driver.close()
                driver.switch_to.window(tabs[0])
                continue


            count += 1

            print("⬇️ Downloading:", pdf_url)

            download(pdf_url, f"law_{count}.pdf")

            driver.close()
            driver.switch_to.window(tabs[0])

            time.sleep(2)

        except Exception as e:
            print("⚠️ Skip:", e)
            driver.switch_to.window(driver.window_handles[0])


driver.quit()

print(f"\n🎉 Finished: {count} files")
